In [ ]:
#On omega with NoMachine
#$ module load fuse
#$ fuse  #if you want to run from the terminal
#$ jupyter-lab # if you want to run from a notebook (preferred)

using Revise
using Plots   # for plotting
using FUSE    # this will also import IMAS in the current namespace

The FUSE flux-matcher has been developed at General Atomics with algorithms based on the TGYRO flux-matcher and an optimizer that is more advanced.

In [ ]:
# Flux-match the DIII-D standard FUSE L-mode case
ini, act = FUSE.case_parameters(:D3D, :L_mode);      # This line can take 15 minutes to run when run for the first time in your project (compilation time)
dd = IMAS.dd()
FUSE.init(dd, ini, act);                         
act.ActorCoreTransport.model = :FluxMatcher;
act.ActorTGLF.warn_nn_train_bounds = false           # print warnings about leaving NN training bounds
act.ActorTGLF.model = :TGLFNN                        # can be :TGLFNN, :TJLF (Julia version of Fortran TGLF), :TGLF (needs GACODE insalled), :QLGYRO (needs GACODE installed)
act.ActorTGLF.onnx_model = false                     # you can run your own PyTorch-trained model saved in ONNX format and placed in TGLFNN repo here: /home/<user>/.julia/dev/TGLFNN/models/
act.ActorTGLF.tglfnn_model = "sat3_em_d3d_azf-1"     # NN model can be symbol (e.g. :sat1), or string of model name (e.g. "sat2_em_d3d_azf-1")
act.ActorTGLF.electromagnetic = true                 # this setting is not relevant for NNs, which have fixed settings from training database

act.ActorFluxMatcher.rho_transport = 0.1:0.05:0.90 # for L-mode and negative triangularity, can try to push to the edge
act.ActorFluxMatcher.max_iterations = 500
act.ActorFluxMatcher.algorithm = :anderson #:simple (best only if monotonically increasing profiles), :anderson, :newton, or :jacobian_based
act.ActorFluxMatcher.step_size = 1.0
act.ActorFluxMatcher.relax = 1.0
act.ActorFluxMatcher.verbose = true
act.ActorFluxMatcher.evolve_rotation = :flux_match        # :flux_match or :fixed
act.ActorFluxMatcher.evolve_pedestal = false         # take experimental pedestal
act.ActorFluxMatcher.evolve_densities = :flux_match  # :flux_match or :fixed

# Show profiles before flux-matcher run
display(act.ActorFluxMatcher)
dd_tglfnn = deepcopy(dd)

#FUSE.ActorPedestal(dd,act)
actor_transport = FUSE.ActorFluxMatcher(dd_tglfnn, act)

if act.ActorTGLF.tglfnn_model == "sat3_em_d3d_azf-1"
    act.ActorTGLF.model = :GKNN
    dd_gknn = deepcopy(dd)
    actor_transport = FUSE.ActorFluxMatcher(dd_gknn, act)
end

display(plot(dd.core_profiles, label=" Experiment"))
display(plot!(dd_tglfnn.core_profiles, label=" TGLF-NN"))

if act.ActorTGLF.tglfnn_model == "sat3_em_d3d_azf-1"
    display(plot!(dd_gknn.core_profiles, label=" GKNN"))
end

# plot the flux_matching
display(plot(dd_tglfnn.core_transport))
if act.ActorTGLF.tglfnn_model == "sat3_em_d3d_azf-1"
    display(plot(dd_gknn.core_transport, label=" GKNN"))
end


In [ ]:
# Flux-match the DIII-D standard FUSE H-mode case
ini, act = FUSE.case_parameters(:D3D, :H_mode);        # This line can take 15 minutes to run when run for the first time in your project (compilation time)
dd = IMAS.dd()
FUSE.init(dd, ini, act);                         
act.ActorCoreTransport.model = :FluxMatcher;
act.ActorTGLF.warn_nn_train_bounds = false             # print warnings about leaving NN training bounds
act.ActorTGLF.model = :TGLFNN                          # can be :TGLFNN, :TJLF (Julia version of Fortran TGLF), :TGLF (needs GACODE insalled), :QLGYRO (needs GACODE installed)
act.ActorTGLF.onnx_model = false                       # you can run your own PyTorch-trained model saved in ONNX format and placed in TGLFNN repo here: /home/<user>/.julia/dev/TGLFNN/models/
act.ActorTGLF.tglfnn_model = "sat3_em_d3d_azf-1"       # NN model can be symbol (e.g. :sat1), or string of model name (e.g. "sat2_em_d3d_azf-1")
act.ActorTGLF.electromagnetic = true                   # this setting is not relevant for NNs, which have fixed settings from training database

act.ActorFluxMatcher.rho_transport = 0.1:0.05:0.85
act.ActorFluxMatcher.max_iterations = 500
act.ActorFluxMatcher.algorithm = :anderson   #:simple (best only if monotonically increasing profiles), :anderson, :newton, or :jacobian_based
act.ActorFluxMatcher.step_size = 0.5
act.ActorFluxMatcher.relax = 1.0
act.ActorFluxMatcher.verbose = true
act.ActorFluxMatcher.evolve_rotation = :flux_match          # :flux_match or :fixed
act.ActorFluxMatcher.evolve_pedestal = false           # take experimental pedestal
act.ActorFluxMatcher.evolve_densities = :flux_match    # :flux_match or :fixed

# Show profiles before flux-matcher run
display(act.ActorFluxMatcher)
dd_tglfnn = deepcopy(dd)

#FUSE.ActorPedestal(dd,act)
actor_transport = FUSE.ActorFluxMatcher(dd_tglfnn, act)
if act.ActorTGLF.tglfnn_model == "sat3_em_d3d_azf-1"
    act.ActorTGLF.model = :GKNN
    dd_gknn = deepcopy(dd)
    actor_transport = FUSE.ActorFluxMatcher(dd_gknn, act)
end

display(plot(dd.core_profiles, label=" Experiment"))
display(plot!(dd_tglfnn.core_profiles, label=" TGLF-NN"))
if act.ActorTGLF.tglfnn_model == "sat3_em_d3d_azf-1"
    display(plot!(dd_gknn.core_profiles, label=" GKNN"))
end

# plot the flux_matching
display(plot(dd_tglfnn.core_transport))
if act.ActorTGLF.tglfnn_model == "sat3_em_d3d_azf-1"
    display(plot(dd_gknn.core_transport))
end


In [ ]:
# You can generate your own ODS using input.gacode and gfile in OMFIT on the omega cluster:
# From file path:
#ig = OMFITinputgacode(os.path.join(inp_dir, filename))
#gfile = OMFITeqdsk(os.path.join(g_dir, gfilename))

# From PROFILES_GEN module:
#ig = OMFIT['PROFILES_GEN_GACODE']['OUTPUTS']['input.gacode']
#gfile = OMFIT['PROFILES_GEN_GACODE']['INPUTS']['gEQDSK']

# Generate ODS:
#ods_ig = ig.to_omas()
#ods_eq = gfile.to_omas()
#ods_ig['equilibrium'] = copy.deepcopy(ods_eq['equilibrium'])
#save_omas_json(ods_ig, "/fusion/projects/omfit-results/<username>/ods_shot_time.json")

In [ ]:
#Start from your own ODS
ods = IMAS.json2imas("/fusion/projects/omfit-results/neisert/ods_166066_4600.json"; show_warnings=false)
ods.core_profiles.profiles_1d[1].time = 4.6
ods.core_profiles.time = [4.6]
ods.core_sources.time = [4.6]

ini, act = FUSE.case_parameters(:D3D, ods);

act.ActorHCD.ec_model = :none
act.ActorHCD.ic_model = :none
act.ActorHCD.lh_model = :none
act.ActorHCD.nb_model = :none
#act.ActorHCD.neutral_model = :none
dd = IMAS.dd()
FUSE.init(dd, ini, act);

act.ActorCoreTransport.model = :FluxMatcher;
act.ActorTGLF.warn_nn_train_bounds = false             # print warnings about leaving NN training bounds
act.ActorTGLF.model = :TGLFNN                          # can be :TGLFNN, :TJLF (Julia version of Fortran TGLF), :TGLF (needs GACODE insalled), :QLGYRO (needs GACODE installed)
act.ActorTGLF.onnx_model = false                       # you can run your own PyTorch-trained model saved in ONNX format and placed in TGLFNN repo here: /home/<user>/.julia/dev/TGLFNN/models/
act.ActorTGLF.tglfnn_model = "sat3_em_d3d_azf-1"       # NN model can be symbol (e.g. :sat1), or string of model name (e.g. "sat2_em_d3d_azf-1")
act.ActorTGLF.electromagnetic = true                   # this setting is not relevant for NNs, which have fixed settings from training database

act.ActorFluxMatcher.rho_transport = 0.1:0.05:0.85
act.ActorFluxMatcher.max_iterations = 500
act.ActorFluxMatcher.algorithm = :anderson     #:simple (best only if monotonically increasing profiles), :anderson, :newton, or :jacobian_based
act.ActorFluxMatcher.step_size = 1.0
act.ActorFluxMatcher.verbose = true
act.ActorFluxMatcher.evolve_rotation = :flux_match     # :flux_match or :fixed
act.ActorFluxMatcher.evolve_pedestal = false           # take experimental pedestal
act.ActorFluxMatcher.evolve_densities = :flux_match    # :flux_match or :fixed

# Show profiles before flux-matcher run
display(act.ActorFluxMatcher)
dd_tglfnn = deepcopy(dd)

#FUSE.ActorPedestal(dd,act)
actor_transport = FUSE.ActorFluxMatcher(dd_tglfnn, act)
if act.ActorTGLF.tglfnn_model == "sat3_em_d3d_azf-1"
    act.ActorTGLF.model = :GKNN
    dd_gknn = deepcopy(dd)
    actor_transport = FUSE.ActorFluxMatcher(dd_gknn, act)
end

display(plot(dd.core_profiles, label=" Experiment"))
display(plot!(dd_tglfnn.core_profiles, label=" TGLF-NN"))
if act.ActorTGLF.tglfnn_model == "sat3_em_d3d_azf-1"
    display(plot!(dd_gknn.core_profiles, label=" GKNN"))
end

# plot the flux_matching
display(plot(dd_tglfnn.core_transport))
if act.ActorTGLF.tglfnn_model == "sat3_em_d3d_azf-1"
    display(plot(dd_gknn.core_transport, label=" GKNN"))
end

In [ ]:
#In order to see what NN models are available, you can prompt the error message giving you all available models:
act.ActorTGLF.tglfnn_model = ""

In [ ]:
# Flux-match the DIII-D standard FUSE L-mode case using TJLF
ini, act = FUSE.case_parameters(:D3D, :L_mode);        # This line can take 15 minutes to run when run for the first time in your project (compilation time)
dd = IMAS.dd()
FUSE.init(dd, ini, act);                         
act.ActorCoreTransport.model = :FluxMatcher;
act.ActorTGLF.warn_nn_train_bounds = false             # print warnings about leaving NN training bounds
act.ActorTGLF.model = :TJLF                            # can be :TGLFNN, :TJLF (Julia version of Fortran TGLF), :TGLF (needs GACODE insalled), :QLGYRO (needs GACODE installed)
act.ActorTGLF.electromagnetic = true                   # this setting is not relevant for NNs, which have fixed settings from training database
act.ActorTGLF.sat_rule = :sat3
act.ActorTGLF.lump_ions = true

act.ActorFluxMatcher.rho_transport = 0.2:0.1:0.8
act.ActorFluxMatcher.max_iterations = 50
act.ActorFluxMatcher.algorithm = :anderson   #:simple (best only if monotonically increasing profiles), :anderson, :newton, or :jacobian_based
act.ActorFluxMatcher.step_size = 1.0
#act.ActorFluxMatcher.relax = 0.5
act.ActorFluxMatcher.verbose = true
act.ActorFluxMatcher.evolve_rotation = :flux_match          # :flux_match or :fixed
act.ActorFluxMatcher.evolve_pedestal = false           # take experimental pedestal
act.ActorFluxMatcher.evolve_densities = :flux_match    # :flux_match or :fixed
#act.ActorNeoclassical.model = :hirshmansigmar

# Show profiles before flux-matcher run
display(act.ActorFluxMatcher)
dd_tjlf = deepcopy(dd)

#FUSE.ActorPedestal(dd,act)
actor_transport = FUSE.ActorFluxMatcher(dd_tjlf, act)

# show after
display(plot(dd.core_profiles, label=" Experiment"))
display(plot!(dd_tjlf.core_profiles, label=" TJLF"))

# plot the flux_matching
display(plot(dd_tjlf.core_transport))

# Use TGLF-NN as preconditioner
act.ActorTGLF.model = :TGLFNN                        
act.ActorTGLF.tglfnn_model = "sat3_em_d3d_azf-1"     
act.ActorFluxMatcher.rho_transport = 0.2:0.1:0.8
act.ActorFluxMatcher.max_iterations = 500
act.ActorFluxMatcher.step_size = 1.0
display(act.ActorFluxMatcher)
dd_tglfnn = deepcopy(dd)
actor_transport = FUSE.ActorFluxMatcher(dd_tglfnn, act)

# Run TJLF
act.ActorTGLF.model = :TJLF
act.ActorFluxMatcher.rho_transport = 0.2:0.1:0.8
act.ActorFluxMatcher.max_iterations = 50
act.ActorFluxMatcher.step_size = 1.0
#act.ActorFluxMatcher.relax = 0.5
dd_tjlf2 = deepcopy(dd_tglfnn)
actor_transport = FUSE.ActorFluxMatcher(dd_tjlf2, act)

# show after
display(plot(dd.core_profiles, label=" Experiment"))
display(plot!(dd_tjlf.core_profiles, label=" TJLF"))
display(plot!(dd_tjlf2.core_profiles, label=" TJLF2"))

# plot the flux_matching
display(plot(dd_tjlf.core_transport))
display(plot(dd_tjlf2.core_transport))

# Compare TJLF2 to GKNN
if act.ActorTGLF.tglfnn_model == "sat3_em_d3d_azf-1"
    act.ActorTGLF.model = :GKNN
    dd_gknn = deepcopy(dd)
    act.ActorFluxMatcher.rho_transport = 0.2:0.1:0.8
    act.ActorFluxMatcher.max_iterations = 300
    act.ActorFluxMatcher.step_size = 1.0
    actor_transport = FUSE.ActorFluxMatcher(dd_gknn, act)
    display(plot(dd.core_profiles, label=" Experiment"))
    display(plot!(dd_gknn.core_profiles, label=" GKNN"))
    display(plot!(dd_tjlf2.core_profiles, label=" TJLF2"))
    display(plot(dd_gknn.core_transport, label=" GKNN"))
end